# Modelo Booleanno de Recuperación de la Información

## Leer m documentos (corpus)

In [111]:
import fitz # open pdf
import re #regex
import unicodedata # use unicode


def clean_text(text):
    # remove accents
    ## decompose unicode glyphs
    normalized_string = unicodedata.normalize('NFKD',  text)
    ## if a glyhp is compose, use its base form
    no_accent_string = ''.join([c for c in normalized_string if not unicodedata.combining(c)])
    # remove punctuation marks
    no_punctuation_string = re.sub(r'[^\w]+', ' ', no_accent_string)
    return no_punctuation_string.strip()

def dictionary(document):
    # strip text of document to only get the main content
    splited_words = re.split(r'[^\w]+|\s+', document)
    terms = []
    for w in splited_words:
        if w not in terms:
            terms.append(w)
    return terms

def read_raw_doc(doc_name):
    doc = fitz.open(doc_name)
    text = []
    for page in doc:
        text.append(page.get_text())
    return ''.join(text)

def clean_doc(doc_name):
    clean_text(read_raw_doc(doc_name))

In [112]:
raw_doc = read_raw_doc("documentos/1984.pdf")
unicode = raw_doc[182:182 + 100]
print(repr(unicode))

'a� en� el� pecho� en� su� esfuerzo� por� burlar� el\nmolestísimo�viento,�se�deslizó�rápidamente�por�e'


In [110]:
for c in unicode:
#    print(f"{c}:{repr(c)}:{ord(c)}")
    pass

def line():
    print('-'*80)

In [108]:
## decompose unicode glyphs
text = unicode
normalized_string = unicodedata.normalize('NFKD',  text)
print(normalized_string)
line()
## if a glyhp is compose, use its base form
no_accent_string = ''.join([c for c in normalized_string if not unicodedata.combining(c)])
print(no_accent_string)
line()
# remove punctuation marks
def p(text, reg):
    no_punctuation_string = re.sub(reg, ' ', text)
    print(no_punctuation_string)
    print(re.split(r'[^\w]+', no_punctuation_string))
    line()

p(no_accent_string, r'[^\w]+')
p(no_accent_string, r"[\x00-\x1F\x7F\uFFFD]")

a� en� el� pecho� en� su� esfuerzo� por� burlar� el
molestísimo�viento,�se�deslizó�rápidamente�por�e
--------------------------------------------------------------------------------
a� en� el� pecho� en� su� esfuerzo� por� burlar� el
molestisimo�viento,�se�deslizo�rapidamente�por�e
--------------------------------------------------------------------------------
a en el pecho en su esfuerzo por burlar el molestisimo viento se deslizo rapidamente por e
['a', 'en', 'el', 'pecho', 'en', 'su', 'esfuerzo', 'por', 'burlar', 'el', 'molestisimo', 'viento', 'se', 'deslizo', 'rapidamente', 'por', 'e']
--------------------------------------------------------------------------------
a  en  el  pecho  en  su  esfuerzo  por  burlar  el molestisimo viento, se deslizo rapidamente por e
['a', 'en', 'el', 'pecho', 'en', 'su', 'esfuerzo', 'por', 'burlar', 'el', 'molestisimo', 'viento', 'se', 'deslizo', 'rapidamente', 'por', 'e']
--------------------------------------------------------------------------

In [87]:
## decompose unicode glyphs
text = unicode
normalized_string = unicodedata.normalize('NFKD',  text)
print(normalized_string)
line()
## if a glyhp is compose, use its base form
no_accent_string = ''.join([c for c in normalized_string if not unicodedata.combining(c)])
print(no_accent_string)
line()
# remove punctuation marks
no_punctuation_string = re.sub(r'[^\w\s]', '', no_accent_string)
print(no_punctuation_string)
line()

a� en� el� pecho� en� su� esfuerzo� por� burlar� el
molestísimo�viento,�se�deslizó�rápidamente�por�e
--------------------------------------------------------------------------------
a� en� el� pecho� en� su� esfuerzo� por� burlar� el
molestisimo�viento,�se�deslizo�rapidamente�por�e
--------------------------------------------------------------------------------
a en el pecho en su esfuerzo por burlar el
molestisimovientosedeslizorapidamentepore
--------------------------------------------------------------------------------


In [46]:
dictionary("\n\n\nHola\n\n\nHola\n\n\n\n")

['', 'Hola', 'Hola', '']

In [22]:
clean_text(" ¿Cómo estás niño? ")

' Como estas nino '

In [ ]:
print(text)

## Generar diccionario de términos de todo el corpus

## Aplicar eliminación de palabras vacías (stopword)

## Aplicar una técnica de stemming para reducir el "Tamaño" de las palabras

## Obtener una matriz binaria de la presencia de los términos en cada documento de todo el corpus

## Diseñar una tabla hash que permita obtener mediante su función. llave -> valor <=> stem -> documentos donde aparece stem

## Leer la consulta booleana Q

### Aplicar stopword y stemming a la consulta Q

### Aplicar la notación postfijo para el procesamiento de recuperación de la consulta dada

## Presentar los nombre de los documentos obtenidos por Q